In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

import base64  #added to display the image



# Call the crud file
from animal_shelter import AnimalShelter

###################################
print("Database loaded")




###########################
# Data Manipulation / Model
###########################
#Login to the database as a user
username = "aacuser"
password = "1234"
shelter = AnimalShelter(username, password)

#################################
print('Test logged in')

# class read method must support return of cursor object 
df=pd.DataFrame.from_records(shelter.read_all({}))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Final Project')

# Add Grazioso Salvare’s logo
image_filename = 'GraziosoSalvareLogo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

####################
print('Test Logo loaded')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.B(html.H1('James Roberts'))),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    html.Hr(),
    #dash_table.DataTable(
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
       
        editable = False,          #non editable
        row_selectable = "single", # select one row
        selected_rows = "Multi",   #select multiple rows
        selected_columns = "Multi",  #select multiple columns
        filter_action = "native", #use a filter
        sort_action = "native", # let the filter sort
        sort_mode = "Multi",   #select more than one option
        row_deletable = False,    # non deletable
        page_action = "native", ##use pagination
        page_current = 0, #set the start page
        page_size = 12, #set rows per page  
    ),
    html.Br(),
    #trying to move my name to the bottom
    html.Center(html.B(html.H1('James Roberts'))),
    html.Hr(),
    html.Div(
        dcc.RadioItems(
        id ='Filter-Type',
        options = [
            {'label': 'Water Rescue', 'value': 'water'},
            {'label': 'Mountain Or Wilderness', 'value': 'wild'},
            {'label': 'Disaster Rescue and Individual search', 'value':'disaster'},
            {'label': 'Reset', 'value': 'All'}
        ],
        )
    ),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])
print('Test filters built')
#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('Filter-Type', 'value')])
def update_dashboard(filter_type):

    #Filter for water rescue, 
        if filter_type == 'water':
            df=pd.DataFrame.from_records(shelter.read_all({
                "animal_type":"Dog",
                "breed": {"$in": ["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]},
                "sex_upon_outcome": "Intact Female",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}  # >or = 26, < kjor = 156
            }))
    #Filter for Mountain rescue
        elif filter_type == "wild":
            df=pd.DataFrame.from_records(shelter.read_all({
                "animal_type":"Dog",
                "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog",
                                  "Siberian Husky,Rottweiler"]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}  # >or = 26, < kjor = 156   
            }))
        
    #Filter for Disaster Rescue
        elif filter_type == "disaster":
            df=pd.DataFrame.from_records(shelter.read_all({
                "animal_type":"Dog",
                "breed": {"$in": ["Doberman Pinscher", "German Shepherd","Golden Retriever", 
                                  "Bloodhound", "Rottweiler"]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":20.0, "$lte":300.0}  # >or = 26, < kjor = 156   
            })) 
    
    #Reset the filters
        else:
            df.pd.DataFrame.from_records(shelter.read_all({}))
        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)



print("Test Filters updated")

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': 'black',
        'color': 'red'
    } for i in selected_columns]
print("Test styles updated")

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    dff = pd.DataFrame.from_dict(viewData)
    #return [
    return[
        dcc.Graph(
            figure = px.histogram(dff, x ='breed')
        )
    ]
 

print("Test Chart built")

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Get the location latitude and longitude columns
    location_lat = dff.columns.get_loc("location_lat")    #Make variable to get latitude
    location_long = dff.columns.get_loc("location_long")  #make variable to get longitude
    # Create a list of markers based on the data
    markers = [
        dl.Marker(
            position=[row[location_lat], row[location_long]], 
            children=[
                dl.Tooltip(row[4]),
                dl.Popup([
                    html.H1(row['name']),      #add animal name to location
                    html.H2(row['animal_id'])  #add animal id to location
                ])
            ]
        )
        for _, row in dff.iterrows()
    ]
    # Create the map with the markers
    return [
        dl.Map(
            style={'width':'1000px','height': '500px'},
            center=[30.75,-97.48],     #set austin as the center of the map.
            zoom=10,
            children=[
                dl.TileLayer(id="base-jlayer-id"),
                *markers
            ]
        )
    ]

    
print("Test Map loaded")
app

Database loaded
Initializing AnimalShelter object...
Connecting to MongoDB with username 'aacuser' and password '1234'...
MongoDB connection successful!
Test logged in
Test Logo loaded
Test filters built
Test Filters updated
Test styles updated
Test Chart built
Test Map loaded
